In [26]:
import json
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI


In [27]:
from dotenv import load_dotenv
import os


load_dotenv()
api_key = os.getenv('APIKEY')

In [28]:
with open('table_info.json', 'r') as file:
    data = json.load(file)

data['base_import_mapping']


{'columns': [{'column_name': 'id', 'data_type': 'integer'},
  {'column_name': 'create_uid', 'data_type': 'integer'},
  {'column_name': 'write_uid', 'data_type': 'integer'},
  {'column_name': 'create_date', 'data_type': 'timestamp without time zone'},
  {'column_name': 'write_date', 'data_type': 'timestamp without time zone'},
  {'column_name': 'field_name', 'data_type': 'character varying'},
  {'column_name': 'res_model', 'data_type': 'character varying'},
  {'column_name': 'column_name', 'data_type': 'character varying'}],
 'primary_keys': ['id'],
 'foreign_keys': [{'foreign_table': 'base_import_mapping',
   'foreign_column': 'create_uid'},
  {'foreign_table': 'base_import_mapping', 'foreign_column': 'write_uid'}]}

In [29]:
print(data.keys())

dict_keys(['base_import_mapping', 'base_import_import', 'base_import_module', 'ir_module_module_dependency', 'bus_bus', 'web_tour_tour', 'bus_presence', 'iap_account', 'iap_account_res_company_rel', 'iap_account_info', 'web_editor_converter_test_sub', 'web_editor_converter_test', 'res_users_settings', 'uom_uom', 'uom_category', 'onboarding_progress', 'onboarding_progress_onboarding_progress_step_rel', 'onboarding_progress_step', 'onboarding_onboarding', 'onboarding_onboarding_onboarding_onboarding_step_rel', 'onboarding_onboarding_step', 'resource_calendar', 'resource_calendar_attendance', 'resource_resource', 'resource_calendar_leaves', 'ir_logging', 'utm_tag', 'ir_module_category', 'ir_module_module', 'utm_stage', 'utm_tag_rel', 'utm_medium', 'utm_source', 'utm_campaign', 'ir_profile', 'calendar_event_res_partner_rel', 'calendar_attendee', 'calendar_recurrence', 'meeting_category_rel', 'calendar_event_type', 'calendar_alarm_calendar_event_rel', 'calendar_filters', 'calendar_provider_

In [30]:
a=data['product_template']['foreign_keys']
b='foreign columns in are '
for det in a:
    b += ' ' + det['foreign_column'] +','
print(b)

foreign columns in are  categ_id, uom_id, uom_po_id, company_id, create_uid, write_uid,


In [31]:
data['product_template']['foreign_keys']

[{'foreign_table': 'product_template', 'foreign_column': 'categ_id'},
 {'foreign_table': 'product_template', 'foreign_column': 'uom_id'},
 {'foreign_table': 'product_template', 'foreign_column': 'uom_po_id'},
 {'foreign_table': 'product_template', 'foreign_column': 'company_id'},
 {'foreign_table': 'product_template', 'foreign_column': 'create_uid'},
 {'foreign_table': 'product_template', 'foreign_column': 'write_uid'}]

In [32]:
rag_data = []
# trying to make a list that contains details of each table in string format
for table in data:
    table_details = ''
    # print(data[table])
    length = len(data[table]['columns'])
    # print(length)
    table_details+= 'Table Name: ' + table + '.' + ' Columns in table ' + table + ' are'
    for col in range(length):
        table_details+= ', ' + data[table]['columns'][col]['column_name']
    table_details += '. Primary keys in the table ' + table + ' are'
    length = len(data[table]['primary_keys'])
    for prim_key in range(length):
        
        table_details += ', ' + data[table]['primary_keys'][prim_key]
    length = len(data[table]['foreign_keys'])
    table_details += '. Foreign keys in the table ' + table + ' are'
    for for_key in range(length):
        table_details += ', ' + data[table]['foreign_keys'][for_key]['foreign_column']
    rag_data.append(table_details)

In [33]:
rag_data[52]

'Table Name: ir_model_constraint. Columns in table ir_model_constraint are, id, model, module, create_uid, write_uid, message, write_date, create_date, name, definition, type. Primary keys in the table ir_model_constraint are, id. Foreign keys in the table ir_model_constraint are, model, module, create_uid, write_uid'

In [34]:
len(rag_data)== len(data.keys()) #Both have same length

True

In [35]:
#Try to create a RAG Model from JSON file that consists the details of SQL

In [36]:
with open('table_info.json', 'r') as file:
    data = json.load(file)

In [37]:
def metadata_func(record: dict, metadata: dict) -> dict:
    columns = record.get("columns", [])
    for_keys = record.get("foreign_keys", [])
    # print(for_keys)
    
    cname = [col.get("column_name") for col in columns]
    dtype = [col.get("data_type") for col in columns]
    f_key = [col.get("foreign_column") for col in for_keys]
    # cdesc = [col.get("Column_Description") for col in columns]

    metadata.update({
        "Table_Name": '',
        # "Table_Description": record.get("Table_Description"),
        "Column_Names": str(cname),
        "Data_Type": str(dtype),
        "Foreign_Key": str(f_key),
        # "Column_Description": str(cdesc),
    })
    return metadata

In [38]:
def load():
    loaded_data  = []
    for table in data:
        metadata = {}
        transformed_data = metadata_func(data[table], metadata)
        transformed_data['Table_Name']=table
        loaded_data.append(transformed_data)
    return loaded_data

In [39]:
data = load()

In [40]:

documents = [
    Document(
        page_content=(
            f"Table Name: {table.get('Table_Name', 'N/A')}. "

        ),
        metadata={
            "Table_Name": table.get("Table_Name", ""),
            "Column_Names": "," + str((table.get("Column_Names", []))),  
            "Data_Type": ", " + str((table.get("Data_Type", []))),        
            "Foreign_Key": ", " + str((table.get("Foreign_Key", [])))
        },
    )
    for table in data
]

In [41]:
documents[0]

Document(metadata={'Table_Name': 'base_import_mapping', 'Column_Names': ",['id', 'create_uid', 'write_uid', 'create_date', 'write_date', 'field_name', 'res_model', 'column_name']", 'Data_Type': ", ['integer', 'integer', 'integer', 'timestamp without time zone', 'timestamp without time zone', 'character varying', 'character varying', 'character varying']", 'Foreign_Key': ", ['create_uid', 'write_uid']"}, page_content='Table Name: base_import_mapping. ')

In [42]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2") #embedding model

In [43]:
vectorstore = Chroma.from_documents(documents, embeddings,persist_directory="./chroma_db") #vector store